## FTIR analysis

In [1]:
from FTIR import *

In [2]:
#read table of rotational constants to pandas df
df=pd.read_csv("tables/const.dat",sep="\s+",header=0,names=["$B_0$", "$B_0err$", "$B_1$", "$B_1err$", "$D_0$", "$D_0err$",
"$D_1$", "$D_1err$", "$\nu_0$", "$\nu_0err$"],index_col=0)
#print(df)
#read in atomic masses used in calculations to data frame. atomic masses from webelements.com
masses=pd.read_csv("tables/amu.dat",sep="\s+",header=0,names=["mass","error"],index_col=0)
print(masses)
m_caption="Masses from webelements.com"
m_label="tab:masses"
masses.to_latex("tables/masses.tex",column_format="lcc",escape=False,caption=m_caption,label=m_label)

           mass         error
H      1.007825  4.000000e-10
D      2.101778  4.000000e-10
35Cl  34.968853  6.900000e-08
37Cl  36.965903  1.100000e-07


In [3]:
#calculate an array of reduced masses and errors for the 4 molecules
mus,dmus=np.zeros(4),np.zeros(4)
k=0
for i in range(2):
    for j in range(2,4):  
        mus[k],dmus[k]=reduced_mass(masses["mass"][i],masses["error"][i],masses["mass"][j],masses["error"][j])
        k+=1

In [11]:
#Use the functions from FTIR.py to calculate the internuclear distances
bls,bls_err=[],[]
be_list,be_err_list=[],[]
for i in range(4):
    Be,Be_err=B_e(df["$B_0$"][i],df["$B_0err$"][i],df["$B_1$"][i],df["$B_1err$"][i])
    be_list=np.append(be_list,Be)
    be_err_list=np.append(be_err_list,Be_err)
    be_err_list
    I,I_err=inertia(Be,Be_err)
    mu,dmu=mus[i],dmus[i]
    bl,bl_err=bond_length(I,I_err,mu,dmu)
    bls=np.append(bls,bl)
    bls_err=np.append(bls_err,bl_err)

indices=["mu","dmu","Be","Be_err","Bond length","Bond length err"]
cols=[r"$H^{35}Cl$",r"$H^{37}Cl$",r"$D^{35}Cl$",r"$D^{37}Cl$"]
decimals = pd.Series([9,8,4,4,4,4], index=indices)
results=pd.DataFrame(np.array([mus,dmus,be_list,be_err_list,bls,bls_err]),columns=cols,index=indices)
#results=results.round(decimals)
results=results.transpose()
head=[r"$\mu$",r"$\Delta\mu$",r"$B_e$",r"$\Delta B_e$",r"$r$",r"$\Delta r$"]
caption="caption"
label=""
results.to_latex("tables/results.tex",header=head,formatters=[formfunc1,formfunc1,formfunc2,formfunc2,formfunc2,formfunc2],column_format="lcccccc",escape=False,caption=caption,label=label)
#print(results)
#sigfig fixed but take mu dmu out into seperate table with 9 sigfig
